In [1]:
import random
model_name = ["albert-base-v1", "facebook/bart-base", "bert-base-uncased", "ctrl", "distilbert-base-uncased", "gpt2", "allenai/longformer-base-4096", "facebook/mbart-large-cc25", "roberta-base", "roberta-large","roberta-large-mnli", "camembert-base", "t5-base", "xlnet-base-cased"]
model_idx = -5


In [2]:
!pip install datasets
!pip install transformers
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [3]:
from transformers import set_seed
set_seed(123)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import datasets

In [6]:
mydata = datasets.load_dataset('csv', data_files='/content/drive/MyDrive/Security/classifier/CF Trainer Data part1.csv')
mydata

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bd2cf2b082283aeb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2401
    })
})

In [7]:
train_dataset, test_dataset= mydata['train'].train_test_split(test_size=0.15).values()
mydata = datasets.DatasetDict({'train': train_dataset, 'test': test_dataset})

In [8]:
mydata["train"][792]

{'text': 'For MGS coding in SVC, two new features have been introduced: motion-compensated prediction for the base layer from the enhancement layer and the supported of so-called key.',
 'label': 0}

In [9]:
from transformers import AutoTokenizer
import sacremoses as sm
tokenizer = AutoTokenizer.from_pretrained(model_name[model_idx])
#tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Security/Pretrained XLNet")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [11]:
tokenized_mydata = mydata.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name[model_idx], num_labels=3)
#model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Security/Pretrained XLNet", num_labels=3)

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

In [14]:
from datasets import load_metric
def custom_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    print(type(eval_pred))
    print(eval_pred)
    logits, labels = np.array(eval_pred)
    print(logits.shape)
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels, average="micro")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="micro")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="micro")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}


In [15]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=50,
    num_train_epochs=15,
    weight_decay=0.01,
    #evaluation_strategy = "epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_mydata["train"],
    eval_dataset=tokenized_mydata["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=custom_metrics
)


In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2040
  Num Epochs = 15
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 50
  Gradient Accumulation steps = 1
  Total optimization steps = 615
  Number of trainable parameters = 355362819
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than usin

Step,Training Loss
500,0.354300


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=615, training_loss=0.2930471241958742, metrics={'train_runtime': 399.334, 'train_samples_per_second': 76.628, 'train_steps_per_second': 1.54, 'total_flos': 5324695203903840.0, 'train_loss': 0.2930471241958742, 'epoch': 15.0})

In [18]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 361
  Batch size = 50


{'eval_loss': 2.295137882232666,
 'eval_runtime': 1.4406,
 'eval_samples_per_second': 250.597,
 'eval_steps_per_second': 5.553,
 'epoch': 15.0}

In [19]:
predictions = trainer.predict(tokenized_mydata["test"])
print(predictions.predictions.shape)
#print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 361
  Batch size = 50


(361, 3)


In [20]:
preds = np.argmax(predictions.predictions, axis=-1)
labels = predictions.label_ids

In [21]:
#!pip install evaluate
#import evaluate

#metric = evaluate.load("glue", "mrpc")
metric1 = load_metric("precision")
metric2 = load_metric("recall")
metric3 = load_metric("f1")
metric4 = load_metric("accuracy")

precision = metric1.compute(predictions=preds, references=labels, average="macro")["precision"]
recall = metric2.compute(predictions=preds, references=labels, average="macro")["recall"]
f1 = metric3.compute(predictions=preds, references=labels, average="macro")["f1"]
accuracy = metric4.compute(predictions=preds, references=labels)["accuracy"]

print("precision "+ str(precision), "recall "+ str(recall), "f1 "+ str(f1), "accuracy "+ str(accuracy))

<ipython-input-21-7bd27732f225>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric1 = load_metric("precision")


precision 0.6042786664011858 recall 0.5884957915873362 f1 0.5953726128168699 accuracy 0.631578947368421


In [22]:
from transformers import pipeline

trained_cf = pipeline(task="text-classification", model=model, tokenizer=tokenizer, device = 'cuda:0')

In [23]:
trained_cf("This also enables authorization of small data delivery by specific AS and prevent unauthorized delivery of MT small data")

[{'label': 'LABEL_1', 'score': 0.9995249509811401}]

In [24]:
np.array((1,2))

array([1, 2])

In [25]:
#!unzip '/content/drive/MyDrive/Security/Pretrained bert/bert_weights.zip' -d '/content/drive/MyDrive/Security/Pretrained bert/'